<a href="https://colab.research.google.com/github/itsMatrik/fish-audio-google-colab-notebook/blob/main/inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fish Speech

# **READ IT PLEASE**

This is fork from original fish-speech (https://github.com/fishaudio/fish-speech). This is google colab notebook version because original repo dont have it. Fully working and ready to use. Just run commands line-by-line. You can get hugginfFace token here: https://huggingface.co/settings/tokens (first you need to create an account and subscribe on s1-mini model on huggingFace model page (its free))

### For Windows User / win用户

In [1]:
!chcp 65001

/bin/bash: line 1: chcp: command not found


### For Linux User / Linux 用户

In [2]:
import locale
locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')

'en_US.UTF-8'

In [3]:
!pip install -q huggingface_hub
import huggingface_hub
print("huggingface_hub version:", huggingface_hub.__version__)

huggingface_hub version: 0.36.0


In [4]:
from getpass import getpass
from huggingface_hub import login
hf_token = getpass("insert huggingFace token: ")
login(token=hf_token)

Вставьте ваш HF токен здесь (скрыто): ··········


In [5]:
!git clone https://github.com/fishaudio/fish-speech.git

Cloning into 'fish-speech'...
remote: Enumerating objects: 6121, done.
remote: Counting objects: 100% (1190/1190), done.
remote: Compressing objects: 100% (286/286), done.
remote: Total 6121 (delta 1040), reused 904 (delta 904), pack-reused 4931 (from 2)
Receiving objects: 100% (6121/6121), 19.77 MiB | 16.08 MiB/s, done.
Resolving deltas: 100% (3984/3984), done.


In [6]:
%cd fish-speech

/content/fish-speech


In [7]:
!apt-get install -y portaudio19-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libasound2-dev libjack-dev libjack0 libportaudio2 libportaudiocpp0
Suggested packages:
  libasound2-doc jackd1 portaudio19-doc
The following packages will be REMOVED:
  libjack-jackd2-0
The following NEW packages will be installed:
  libasound2-dev libjack-dev libjack0 libportaudio2 libportaudiocpp0
  portaudio19-dev
0 upgraded, 6 newly installed, 1 to remove and 41 not upgraded.
Need to get 596 kB of archives.
After this operation, 3,178 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libjack0 amd64 1:0.125.0-3build2 [93.3 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libasound2-dev amd64 1.2.6.1-1ubuntu1 [110 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libjack-dev amd64 1:0.125.0-3build2 [206 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/univers

In [8]:
!pip install -e .

Obtaining file:///content/fish-speech
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 5.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.6/40.6 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 14.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 7.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 

In [9]:
!python -c "import sys; import torch, torchaudio; print('python', sys.version.split()[0]); print('torch', getattr(torch,'__version__',None)); print('torchaudio', getattr(torchaudio,'__version__',None))"

python 3.12.12
torch 2.9.0+cu126
torchaudio 2.9.0+cu126


In [10]:
!pip uninstall -y torchaudio torch

Found existing installation: torchaudio 2.9.0+cu126
Uninstalling torchaudio-2.9.0+cu126:
  Successfully uninstalled torchaudio-2.9.0+cu126
Found existing installation: torch 2.9.0+cu126
Uninstalling torch-2.9.0+cu126:
  Successfully uninstalled torch-2.9.0+cu126


In [11]:
!pip install --force-reinstall torch==2.6.0+cu124 torchaudio==2.6.0+cu124 --index-url https://download.pytorch.org/whl/cu124

Looking in indexes: https://download.pytorch.org/whl/cu124
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 90.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 63.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 93.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1

In [13]:
!pip uninstall -y torchvision torchmetrics

Found existing installation: torchvision 0.24.0+cu126
Uninstalling torchvision-0.24.0+cu126:
  Successfully uninstalled torchvision-0.24.0+cu126
Found existing installation: torchmetrics 1.8.2
Uninstalling torchmetrics-1.8.2:
  Successfully uninstalled torchmetrics-1.8.2


In [14]:
!pip install --force-reinstall torchvision==0.21.0+cu124 --index-url https://download.pytorch.org/whl/cu124

Looking in indexes: https://download.pytorch.org/whl/cu124
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 6.3 MB/s eta 0:00:00
  Using cached https://download.pytorch.org/whl/cu124/torch-2.6.0%2Bcu124-cp312-cp312-linux_x86_64.whl.metadata (28 kB)
  Using cached filelock-3.20.0-py3-none-any.whl.metadata (2.1 kB)
  Using cached https://download.pytorch.org/whl/typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached networkx-3.6.1-py3-none-any.whl.metadata (6.8 kB)
  Using cached https://download.pytorch.org/whl/jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached fsspec-2025.12.0-py3-none-any.whl.metadata (10 kB)
  Using cached https://download.pytorch.org/whl/cu124/nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl (24.6 MB)
  Using cached https://download.pytorch.org/whl/cu124/nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl (883 kB)
  Using cached https://download.pytorch.org/whl/cu124/nvidia_cuda_cupti_cu12-12

In [15]:
!pip install --no-cache-dir --upgrade "torchmetrics==1.8.2"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 206.6 MB/s eta 0:00:00


### Prepare Model

In [16]:
# For Chinese users, you probably want to use mirror to accelerate downloading
# !set HF_ENDPOINT=https://hf-mirror.com
# !export HF_ENDPOINT=https://hf-mirror.com

!hf download fishaudio/openaudio-s1-mini --local-dir checkpoints/openaudio-s1-mini/

Fetching 7 files:   0% 0/7 [00:00<?, ?it/s]Downloading 'special_tokens.json' to 'checkpoints/openaudio-s1-mini/.cache/huggingface/download/Pdr1pnDFqf3r8xSTD-lPnaCpeRA=.954449504ac94d1a89b31aa1bc914f07f898b6bb.incomplete'

.gitattributes: 100% 1.52k/1.52k [00:00<00:00, 9.82MB/s]

README.md: 100% 2.86k/2.86k [00:00<00:00, 23.9MB/s]
Download complete. Moving file to checkpoints/openaudio-s1-mini/.gitattributes
Download complete. Moving file to checkpoints/openaudio-s1-mini/README.md
Fetching 7 files:  14% 1/7 [00:00<00:02,  2.02it/s]
special_tokens.json:   0% 0.00/126k [00:00<?, ?B/s]

tokenizer.tiktoken:   0% 0.00/2.56M [00:00<?, ?B/s]


config.json: 100% 844/844 [00:00<00:00, 9.39MB/s]
Download complete. Moving file to checkpoints/openaudio-s1-mini/config.json
special_tokens.json: 100% 126k/126k [00:00<00:00, 1.39MB/s]
Download complete. Moving file to checkpoints/openaudio-s1-mini/special_tokens.json

model.pth:   0% 0.00/1.74G [00:00<?, ?B/s]


codec.pth:   0% 0.00/1.87G [00:00<?, ?B/

## WebUI Inference

> You can use --compile to fuse CUDA kernels for faster inference (10x).

In [17]:
import os
os.environ['GRADIO_SHARE'] = 'True'

!python tools/run_webui.py \
  --llama-checkpoint-path checkpoints/openaudio-s1-mini \
  --decoder-checkpoint-path checkpoints/openaudio-s1-mini/codec.pth \
  --device cuda \
  --half \
  --compile

2026-02-01 20:44:45.533234: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1769978685.819884    4932 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1769978685.894814    4932 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1769978686.460908    4932 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769978686.460955    4932 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769978686.460960    4932 computation_placer.cc:177] computation placer alr

## Break-down CLI Inference

### 1. Encode reference audio: / 从语音生成 prompt:

You should get a `fake.npy` file.

你应该能得到一个 `fake.npy` 文件.

In [ ]:
## Enter the path to the audio file here
src_audio = r"D:\PythonProject\vo_hutao_draw_appear.wav"

!python fish_speech/models/dac/inference.py \
    -i {src_audio} \
    --checkpoint-path "checkpoints/openaudio-s1-mini/codec.pth"

from IPython.display import Audio, display
audio = Audio(filename="fake.wav")
display(audio)

### 2. Generate semantic tokens from text: / 从文本生成语义 token:

> This command will create a codes_N file in the working directory, where N is an integer starting from 0.

> You may want to use `--compile` to fuse CUDA kernels for faster inference (~30 tokens/second -> ~300 tokens/second).

> 该命令会在工作目录下创建 codes_N 文件, 其中 N 是从 0 开始的整数.

> 您可以使用 `--compile` 来融合 cuda 内核以实现更快的推理 (~30 tokens/秒 -> ~300 tokens/秒)

In [ ]:
!python fish_speech/models/text2semantic/inference.py \
    --text "hello world" \
    --prompt-text "The text corresponding to reference audio" \
    --prompt-tokens "fake.npy" \
    --checkpoint-path "checkpoints/openaudio-s1-mini" \
    --num-samples 2
    # --compile

### 3. Generate speech from semantic tokens: / 从语义 token 生成人声:

In [ ]:
!python fish_speech/models/dac/inference.py \
    -i "codes_0.npy" \
    --checkpoint-path "checkpoints/openaudio-s1-mini/codec.pth"

from IPython.display import Audio, display
audio = Audio(filename="fake.wav")
display(audio)